Documentation: https://docs.xarray.dev/en/latest/getting-started-guide/index.html

Helpful github pages/ online textbook : https://earth-env-data-science.github.io/lectures/xarray/xarray.html#broadcasting-aligment-and-combining-data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

1. Load all datasets


In [ ]:
%ls '/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets'

 annual_maxhuss_dataset.nc    annual_minhuss_dataset.nc
 annual_maxprec_dataset.nc    annual_minprec_dataset.nc
 annual_maxtemp_dataset.nc    annual_mintemp_dataset.nc
 annual_meanhuss_dataset.nc  'Copy of 2008-2023 WHS EVENTS WITH LOCATIONS.csv'
 annual_meanprec_dataset.nc  'Copy of  2013-2022.csv'
 annual_meanrsds_dataset.nc   map_data.csv
 annual_mean_temp.nc          map_data.gsheet


In [ ]:
%ls /content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/uscounties.csv

/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/uscounties.csv


In [ ]:
ds_meanTemp = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_mean_temp.nc')
ds_maxTemp = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_maxtemp_dataset.nc')
ds_minTemp = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_mintemp_dataset.nc')
ds_meanHuss = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_meanhuss_dataset.nc')
ds_maxHuss = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_maxhuss_dataset.nc')
ds_minHuss = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_minhuss_dataset.nc')
ds_meanPrec = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_meanprec_dataset.nc')
ds_maxPrec = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_maxprec_dataset.nc')
ds_minPrec = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_minprec_dataset.nc')
ds_meanRsds = xr.open_dataset('/content/drive/Shareddrives/BioInterphase Bat Data Sets/Team B data sets/annual_meanrsds_dataset.nc')



df_bat_data = pd.read_csv('/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/standardized_merged_data.csv')
df_counties = pd.read_csv('/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/uscounties.csv')


In [ ]:
ds_meanTemp = ds_meanTemp.rename({'temp': 'meanTemp'}) #renaming variables so that I can merge into a dataset with these datasets as separate variables
ds_maxTemp = ds_maxTemp.rename({'temp': 'maxTemp'})
ds_minTemp = ds_minTemp.rename({'temp': 'minTemp'})
ds_meanHuss = ds_meanHuss.rename({'huss': 'meanHuss'}) #renaming variables so that I can merge into a dataset with these datasets as separate variables
ds_maxHuss = ds_maxHuss.rename({'huss': 'maxHuss'})
ds_minHuss = ds_minHuss.rename({'huss': 'minHuss'})
ds_meanPrec = ds_meanPrec.rename({'prec': 'meanPrec'}) #renaming variables so that I can merge into a dataset with these datasets as separate variables
ds_maxPrec = ds_maxPrec.rename({'prec': 'maxPrec'})
ds_minPrec = ds_minPrec.rename({'prec': 'minPrec'})
ds_meanRsds = ds_meanRsds.rename({'rsds': 'meanRsds'})

In [ ]:
merged_clim_da = xr.combine_by_coords([ds_meanTemp, ds_maxTemp, ds_minTemp, ds_meanHuss, ds_maxHuss, ds_minHuss, ds_meanPrec, ds_maxPrec, ds_minPrec, ds_meanRsds]) #add extra ds here inside brackets [] as we load other features such as humidity (ds_humidity) and elevation

print(merged_clim_da) #note that Data variables now has maxTemp and meanTemp

<xarray.Dataset>
Dimensions:   (lat: 258, lon: 600, year: 95)
Coordinates:
  * lat       (lat) float64 12.12 12.38 12.62 12.88 ... 75.62 75.88 76.12 76.38
  * lon       (lon) float64 -171.9 -171.6 -171.4 -171.1 ... -22.62 -22.38 -22.12
  * year      (year) int64 2006 2007 2008 2009 2010 ... 2096 2097 2098 2099 2100
Data variables:
    maxHuss   (year, lat, lon) float32 ...
    maxPrec   (year, lat, lon) float32 ...
    maxTemp   (year, lat, lon) float32 ...
    meanHuss  (year, lat, lon) float32 ...
    meanPrec  (year, lat, lon) float32 ...
    meanRsds  (year, lat, lon) float32 ...
    meanTemp  (year, lat, lon) float32 ...
    minHuss   (year, lat, lon) float32 ...
    minPrec   (year, lat, lon) float32 ...
    minTemp   (year, lat, lon) float32 ...


In [ ]:
df_counties['county'] = df_counties['county'].str.lower()

**2**. Merge Bat data with counties to a CSV with counties and their long and lat

In [ ]:
#df_bat_data
df_bat_data = df_bat_data[df_bat_data['county'] != 'redacted']
df_bat_data = df_bat_data[df_bat_data['county'] != 'multiple']
#'multiple'
#first i removed any rows with redacted county, i cannot specify specfic enough lat/long coordinates without that value
#this ends up being only one row

In [ ]:
# Check for NaN values in any column
#df_bat_data.isnull().any()

renamed_counties = {
    'de kalb': 'dekalb',
    'comanmadeche': 'comanche',
    'st louis' : 'st. louis',
    'hardemanmade' : 'hardeman',
    'skamanmadeia' : 'skamania',
    'st croix' : 'st. croix'

}
df_bat_data['county'] = df_bat_data['county'].replace(renamed_counties)

In [ ]:
state_mapping = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
df_bat_data['state'] = df_bat_data['state'].replace(state_mapping)
#df_bat_data.isnull().any() #check that state and county dont have null values

In [ ]:
df_bat_data

,label,country,state,county,Event Start Date,Site.Ownership,Site.Access,Site.Classification,Site.Use,Total.Live,Total.Dead,Total.Bat.Count,vulnerable_c,vulnerable_p,Total.Species,h.density50,type
0,PD,United States,AL,limestone,2014-15,Private,Open-seasonal/restricted,Cave-undeveloped,Hibernaculum,0,0,0,0.0,0.0,0.0,47.471111,cave
1,PD,United States,AL,dekalb,2015-16,Private,Open-all year,Cave-undeveloped,Hibernaculum,71,0,71,71.0,1.0,1.0,29.963640,cave
2,PD,United States,AL,franklin,2015-16,Public,Open-all year,Cave-undeveloped,Hibernaculum,6,0,6,6.0,1.0,1.0,13.917683,cave
3,PD,United States,AL,shelby,2016-17,Public,Open-all year,Cave-undeveloped,Hibernaculum,295,0,295,51.0,1.0,3.0,121.041336,cave
4,NEG,United States,AL,colbert,2014-15,Private,Open-all year,Cave-undeveloped,Hibernaculum,600,0,600,600.0,1.0,1.0,22.838100,cave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,WNS,United States,ME,oxford,2010-11,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN
1677,NEG,United States,ME,piscataquis,2012-13,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN
1678,NEG,United States,WI,iron,2015-16,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN
1679,WNS,United States,WI,douglas,2015-16,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_counties

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population
0,los angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3209,-118.2247,10019635
1,cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5265398
2,harris,Harris,Harris County,48201,TX,Texas,29.8577,-95.3936,4697957
3,maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4367186
4,san diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7350,3296317
...,...,...,...,...,...,...,...,...,...
3138,blaine,Blaine,Blaine County,31009,NE,Nebraska,41.9128,-99.9767,346
3139,king,King,King County,48269,TX,Texas,33.6165,-100.2558,229
3140,kenedy,Kenedy,Kenedy County,48261,TX,Texas,26.9285,-97.7017,169
3141,loving,Loving,Loving County,48301,TX,Texas,31.8493,-103.5800,83


In [ ]:
#warning about counties data set is that they include some cities? which seem to be county seats?

In [ ]:
# Remove duplicates of county, state pairs  #I assume it keeps first occurence, which should be fine since this is ordered by higher population and the county vs city means county should come first
df_counties = df_counties.drop_duplicates(subset=['state_id', 'county']) #for some reason there is duplicate of fairfax, VA with two population counts?

In [ ]:
#add lat and long values based on match of county and state to bat data
merged_da = pd.merge(df_bat_data, df_counties,  how='left', left_on=['state','county'], right_on = ['state_id','county'])

In [ ]:
merged_da

,label,country,state,county,Event Start Date,Site.Ownership,Site.Access,Site.Classification,Site.Use,Total.Live,...,h.density50,type,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population
0,PD,United States,AL,limestone,2014-15,Private,Open-seasonal/restricted,Cave-undeveloped,Hibernaculum,0,...,47.471111,cave,Limestone,Limestone County,1083.0,AL,Alabama,34.8101,-86.9814,101217.0
1,PD,United States,AL,dekalb,2015-16,Private,Open-all year,Cave-undeveloped,Hibernaculum,71,...,29.963640,cave,DeKalb,DeKalb County,1049.0,AL,Alabama,34.4598,-85.8041,71554.0
2,PD,United States,AL,franklin,2015-16,Public,Open-all year,Cave-undeveloped,Hibernaculum,6,...,13.917683,cave,Franklin,Franklin County,1059.0,AL,Alabama,34.4417,-87.8437,32034.0
3,PD,United States,AL,shelby,2016-17,Public,Open-all year,Cave-undeveloped,Hibernaculum,295,...,121.041336,cave,Shelby,Shelby County,1117.0,AL,Alabama,33.2643,-86.6607,220780.0
4,NEG,United States,AL,colbert,2014-15,Private,Open-all year,Cave-undeveloped,Hibernaculum,600,...,22.838100,cave,Colbert,Colbert County,1033.0,AL,Alabama,34.7005,-87.8049,56789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,WNS,United States,ME,oxford,2010-11,NaN,NaN,NaN,NaN,0,...,NaN,NaN,Oxford,Oxford County,23017.0,ME,Maine,44.4999,-70.7566,57807.0
1675,NEG,United States,ME,piscataquis,2012-13,NaN,NaN,NaN,NaN,0,...,NaN,NaN,Piscataquis,Piscataquis County,23021.0,ME,Maine,45.8373,-69.2846,16760.0
1676,NEG,United States,WI,iron,2015-16,NaN,NaN,NaN,NaN,0,...,NaN,NaN,Iron,Iron County,55051.0,WI,Wisconsin,46.2623,-90.2420,6074.0
1677,WNS,United States,WI,douglas,2015-16,NaN,NaN,NaN,NaN,0,...,NaN,NaN,Douglas,Douglas County,55031.0,WI,Wisconsin,46.4329,-91.9162,44034.0


In [ ]:
#merged_da['lat'].isnull()

#print(merged_da[merged_da['lat'].isnull().any()])
merged_da[merged_da['lat'].isnull()]
#now all the canada ones dont have long lat, for now we will drop so I can see how xarray can be handled with variables
merged_da = merged_da[merged_da['country'] != 'Canada']



In [ ]:
merged_da['year'] = merged_da['Event Start Date'].str.split('-').str[0]
merged_da['lon'] = merged_da['lng']

<ipython-input-44-5b2adc6bf41c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_da['year'] = merged_da['Event Start Date'].str.split('-').str[0]
<ipython-input-44-5b2adc6bf41c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_da['lon'] = merged_da['lng']


In [ ]:
merged_da.columns

Index(['label', 'country', 'state', 'county', 'Event Start Date',
       'Site.Ownership', 'Site.Access', 'Site.Classification', 'Site.Use',
       'Total.Live', 'Total.Dead', 'Total.Bat.Count', 'vulnerable_c',
       'vulnerable_p', 'Total.Species', 'h.density50', 'type', 'county_ascii',
       'county_full', 'county_fips', 'state_id', 'state_name', 'lat', 'lng',
       'population', 'year', 'lon'],
      dtype='object')

In [ ]:
merged_da

,label,country,state,county,Event Start Date,Site.Ownership,Site.Access,Site.Classification,Site.Use,Total.Live,...,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,year,lon
0,PD,United States,AL,limestone,2014-15,Private,Open-seasonal/restricted,Cave-undeveloped,Hibernaculum,0,...,Limestone,Limestone County,1083.0,AL,Alabama,34.8101,-86.9814,101217.0,2014,-86.9814
1,PD,United States,AL,dekalb,2015-16,Private,Open-all year,Cave-undeveloped,Hibernaculum,71,...,DeKalb,DeKalb County,1049.0,AL,Alabama,34.4598,-85.8041,71554.0,2015,-85.8041
2,PD,United States,AL,franklin,2015-16,Public,Open-all year,Cave-undeveloped,Hibernaculum,6,...,Franklin,Franklin County,1059.0,AL,Alabama,34.4417,-87.8437,32034.0,2015,-87.8437
3,PD,United States,AL,shelby,2016-17,Public,Open-all year,Cave-undeveloped,Hibernaculum,295,...,Shelby,Shelby County,1117.0,AL,Alabama,33.2643,-86.6607,220780.0,2016,-86.6607
4,NEG,United States,AL,colbert,2014-15,Private,Open-all year,Cave-undeveloped,Hibernaculum,600,...,Colbert,Colbert County,1033.0,AL,Alabama,34.7005,-87.8049,56789.0,2014,-87.8049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,WNS,United States,ME,oxford,2010-11,NaN,NaN,NaN,NaN,0,...,Oxford,Oxford County,23017.0,ME,Maine,44.4999,-70.7566,57807.0,2010,-70.7566
1675,NEG,United States,ME,piscataquis,2012-13,NaN,NaN,NaN,NaN,0,...,Piscataquis,Piscataquis County,23021.0,ME,Maine,45.8373,-69.2846,16760.0,2012,-69.2846
1676,NEG,United States,WI,iron,2015-16,NaN,NaN,NaN,NaN,0,...,Iron,Iron County,55051.0,WI,Wisconsin,46.2623,-90.2420,6074.0,2015,-90.2420
1677,WNS,United States,WI,douglas,2015-16,NaN,NaN,NaN,NaN,0,...,Douglas,Douglas County,55031.0,WI,Wisconsin,46.4329,-91.9162,44034.0,2015,-91.9162


In [ ]:

reference_lat_coordinates = merged_clim_da.coords['lat'].values
reference_lng_coordinates = merged_clim_da.coords['lon'].values
merged_da = merged_da.rename(columns={'lat': 'lat_county'})
merged_da = merged_da.rename(columns={'lng': 'lng_county'})
merged_da['lat'] = merged_da['lat_county'].apply(lambda x: reference_lat_coordinates[np.argmin(np.abs(reference_lat_coordinates - x))])
merged_da['lon'] = merged_da['lng_county'].apply(lambda x: reference_lng_coordinates[np.argmin(np.abs(reference_lng_coordinates - x))])

3. Drop all irrelevant columns

In [ ]:
columns_to_keep = ['label', 'lat', 'lon', 'year'] #keep the columns for coordinates and labels
sub_merged_da = merged_da[columns_to_keep]
sub_merged_da

,label,lat,lon,year
0,PD,34.875,-86.875,2014
1,PD,34.375,-85.875,2015
2,PD,34.375,-87.875,2015
3,PD,33.375,-86.625,2016
4,NEG,34.625,-87.875,2014
...,...,...,...,...
1674,WNS,44.375,-70.875,2010
1675,NEG,45.875,-69.375,2012
1676,NEG,46.375,-90.125,2015
1677,WNS,46.375,-91.875,2015


1. May have to forecast variables in time: lost information? for ex: if wns detected in 2013 at a certain coordinate.. it is possible that there isnt a label in 2014. for wns detection
2. Maybe create binary indicators for each label? Then we can forecast that forward..
3.

4. Add as an array with Xarray climate data, so it is a "variable" that can be accessed with coordinates

In [ ]:
sub_merged_da = sub_merged_da.drop_duplicates()

In [ ]:
sub_merged_da['year'] = pd.to_datetime(sub_merged_da['year']).dt.year
sub_merged_da['lat'].dtypes

sub_merged_da = sub_merged_da.reset_index()
sub_merged_da

<ipython-input-50-aef23ba82b90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_merged_da['year'] = pd.to_datetime(sub_merged_da['year']).dt.year


,index,label,lat,lon,year
0,0,PD,34.875,-86.875,2014
1,1,PD,34.375,-85.875,2015
2,2,PD,34.375,-87.875,2015
3,3,PD,33.375,-86.625,2016
4,4,NEG,34.625,-87.875,2014
...,...,...,...,...,...
1101,1669,WNS,44.625,-72.625,2009
1102,1671,WNS,44.625,-74.375,2008
1103,1672,WNS,44.625,-73.625,2008
1104,1673,WNS,44.625,-71.375,2009


In [ ]:
# Define the shape and coordinates for the null variable
null_variable_shape = (merged_clim_da.dims['lat'], merged_clim_da.dims['lon'], merged_clim_da.dims['year'])
null_variable_coords = merged_clim_da.coords  # Use the same coordinates as an existing variable

# Create the null variable
#null_variable = xr.DataArray(np.nan * np.empty(null_variable_shape), coords=null_variable_coords)
null_variable_data = np.empty(null_variable_shape, dtype=str)
null_variable = xr.DataArray(null_variable_data, coords=null_variable_coords, dims=['lat', 'lon', 'year'])

# Add the null variable to the dataset
merged_clim_da['null_variable'] = null_variable

# Print the updated dataset
merged_clim_da




<xarray.Dataset>
Dimensions:        (lat: 258, lon: 600, year: 95)
Coordinates:
  * lat            (lat) float64 12.12 12.38 12.62 12.88 ... 75.88 76.12 76.38
  * lon            (lon) float64 -171.9 -171.6 -171.4 ... -22.62 -22.38 -22.12
  * year           (year) int64 2006 2007 2008 2009 2010 ... 2097 2098 2099 2100
Data variables:
    maxHuss        (year, lat, lon) float32 ...
    maxPrec        (year, lat, lon) float32 ...
    maxTemp        (year, lat, lon) float32 ...
    meanHuss       (year, lat, lon) float32 ...
    meanPrec       (year, lat, lon) float32 ...
    meanRsds       (year, lat, lon) float32 ...
    meanTemp       (year, lat, lon) float32 ...
    minHuss        (year, lat, lon) float32 ...
    minPrec        (year, lat, lon) float32 ...
    minTemp        (year, lat, lon) float32 ...
    null_variable  (lat, lon, year) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''

In [ ]:
# Replace NaN values in the null variable with corresponding values from the DataFrame
df_with_values = sub_merged_da


# Fill in the null variable in the xarray Dataset with values from the DataFrame
for i in range(len(df_with_values)):
    row = df_with_values.iloc[i]
    merged_clim_da['null_variable'].loc[{'lat': row['lat'], 'lon': row['lon'], 'year': row['year']}] = row['label']



In [ ]:
merged_clim_da

<xarray.Dataset>
Dimensions:        (lat: 258, lon: 600, year: 95)
Coordinates:
  * lat            (lat) float64 12.12 12.38 12.62 12.88 ... 75.88 76.12 76.38
  * lon            (lon) float64 -171.9 -171.6 -171.4 ... -22.62 -22.38 -22.12
  * year           (year) int64 2006 2007 2008 2009 2010 ... 2097 2098 2099 2100
Data variables:
    maxHuss        (year, lat, lon) float32 ...
    maxPrec        (year, lat, lon) float32 ...
    maxTemp        (year, lat, lon) float32 ...
    meanHuss       (year, lat, lon) float32 ...
    meanPrec       (year, lat, lon) float32 ...
    meanRsds       (year, lat, lon) float32 ...
    meanTemp       (year, lat, lon) float32 ...
    minHuss        (year, lat, lon) float32 ...
    minPrec        (year, lat, lon) float32 ...
    minTemp        (year, lat, lon) float32 ...
    null_variable  (lat, lon, year) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''

In [ ]:
file_path = '/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/final_merged_dataset.nc'
merged_clim_da.to_netcdf(file_path)

In [ ]:
#unique_values = np.unique(merged_clim_da['null_variable'].values)
#unique_values

In [ ]:
#for i in range(70,90,1):
#  for j in range(330,350,1):
#    print(merged_clim_da.null_variable.values[i][j]) #might have to cast white nose syndrome variable forward